In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
ROOT = r"D:\02PHBS_G2\PHBS_m2\bigDataAnalysis\big-data-hft\project"
os.chdir(ROOT)

In [9]:
ref_morning_time = pd.date_range("9:31", "11:30", freq="1min").time
ref_afternoon_time = pd.date_range("13:01", "15:00", freq="1min").time

ref_time_arr = np.concatenate([ref_morning_time,ref_afternoon_time])

ref_time_str_list = [x.strftime("%H%M") for x in ref_time_arr]

In [10]:
def my_fillna(df):
    df[['Close']] = df[['Close']].fillna(method='ffill')
    df[['Close','High','Low','Open']] = df[['Close','High','Low','Open']].fillna(method='ffill',axis=1)
    df[['Amount','Count','Qty']] = df[['Amount','Count','Qty']].fillna(0)
    return df

In [22]:
def process_mr_data(path):
    raw_data = pd.read_csv(path, header=None)
    raw_data.columns = ['raw']
    raw_data = raw_data[raw_data["raw"].str.startswith("6") ]
    tmp1 = raw_data["raw"].str.split("#")
    raw_data["stock_code"] = tmp1.str[0]
    tmp2 = tmp1.str[1].str.split("$")
    raw_data["time_str"] = tmp2.str[0].str[8:]
    tmp3 = tmp2.str[1].str.split("\t")
    raw_data["feature_name"] = tmp3.str[0]
    raw_data["feature_value"] = tmp3.str[1].astype("float32")
    data = raw_data.iloc[:,1:]
    min_data = data.pivot(index=['stock_code','time_str'],columns='feature_name',values='feature_value')
    ref_code_list = min_data.index.unique(0)
    ref_code_time_index =pd.MultiIndex.from_product([ref_code_list,ref_time_str_list],names=['stock_code','time_str'])
    min_data_reindex = min_data.reindex(index=ref_code_time_index)
    min_data_final = min_data_reindex.groupby("stock_code").apply(my_fillna)
    min_data_final['Return'] = min_data_final['Close']/min_data_final['Open'] - 1
    return min_data_final

In [23]:
raw_mr_data_dir = "./data/rawDataFromMR/"
cleaned_mins_data_dir = "./data/min_data_cleaned/"
for date_csv in os.listdir(raw_mr_data_dir):
    data_path = os.path.join(raw_mr_data_dir, date_csv,date_csv,date_csv)
    min_data_final = process_mr_data(data_path)
    min_data_final.to_csv(os.path.join(cleaned_mins_data_dir,date_csv))
    print(date_csv)

20190107.csv
20190108.csv
20190109.csv
20190110.csv
20190111.csv
20190114.csv
20190115.csv
20190116.csv
20190117.csv
20190118.csv
20190121.csv
20190123.csv
20190201.csv
20190211.csv
20190212.csv
20190213.csv
20190214.csv
20190215.csv
20190218.csv
20190219.csv
20190220.csv
20190221.csv
20190222.csv
20190225.csv
20190226.csv


raw data preview

In [25]:
pd.read_csv(data_path, header=None)

,0
0,010107#201902260925$Amount\t46417.5
1,010107#201902260925$Close\t103.15
2,010107#201902260925$Count\t4.0
3,010107#201902260925$High\t103.15
4,010107#201902260925$Low\t103.15
...,...
2715302,900957#201902261500$Count\t13.0
2715303,900957#201902261500$High\t0.676
2715304,900957#201902261500$Low\t0.676
2715305,900957#201902261500$Open\t0.676


cleaned data preview

In [27]:
cleaned_data = pd.read_csv(os.path.join(clearned_mins_data_dir,date_csv),index_col=[0,1])
cleaned_data

Amount  Close  Count   High    Low   Open        Qty  \
stock_code time_str                                                             
600000     931       15748574.0  11.87  525.0  11.89  11.85  11.88  1326100.0   
           932       26250540.0  11.95  756.0  11.95  11.87  11.87  2205393.0   
           933       19444028.0  12.07  759.0  12.12  11.92  11.93  1617667.0   
           934       19573332.0  12.09  639.0  12.10  12.06  12.08  1619906.0   
           935       19664888.0  12.13  774.0  12.13  12.07  12.09  1626043.0   
...                         ...    ...    ...    ...    ...    ...        ...   
603999     1456       1368014.0   5.89   89.0   5.90   5.88   5.90   232400.0   
           1457             0.0   5.89    0.0   5.89   5.89   5.89        0.0   
           1458             0.0   5.89    0.0   5.89   5.89   5.89        0.0   
           1459             0.0   5.89    0.0   5.89   5.89   5.89        0.0   
           1500       1237152.0   5.88  147.0   5.88   5.88   5.88   210400.0   

                       Return  
stock_code time_str            
600000     931      -0.000842  
           932       0.006740  
           933       0.011735  
           934       0.000828  
           935       0.003309  
...                       ...  
603999     1456     -0.001695  
           1457      0.000000  
           1458      0.000000  
           1459      0.000000  
           1500      0.000000  

[347760 rows x 8 columns]

In [36]:
tmp = pd.read_pickle(r"D:\02PHBS_G2\PHBS_m2\bigDataAnalysis\big-data-hft\project\data\factor\tickcount_entropy_1m_l1.pkl",compression='gzip')

In [46]:
tmp.to_csv(r"D:\02PHBS_G2\PHBS_m2\bigDataAnalysis\big-data-hft\project\data\factor\tickcount_entropy_1m_l1.csv")

In [49]:
tmp=pd.read_csv(r"D:\02PHBS_G2\PHBS_m2\bigDataAnalysis\big-data-hft\project\data\factor\tickcount_entropy_1m_l1.csv",index_col=0)

In [50]:
tmp

,600373.SH,300557.SZ,000416.SZ,002156.SZ,600500.SH,002123.SZ,000610.SZ,601699.SH,603336.SH,000663.SZ,...,000581.SZ,000606.SZ,600335.SH,600010.SH,300582.SZ,600527.SH,000699.SZ,002011.SZ,600129.SH,002233.SZ
20170103,5.251086,5.053599,4.867754,NaN,5.224017,5.257858,5.114219,5.213046,5.167667,5.183745,...,5.369580,5.024084,5.338118,4.733854,NaN,5.186085,NaN,5.241605,5.105444,5.109035
20170104,5.263410,5.177236,5.289164,NaN,5.331419,5.196428,5.214777,5.149574,5.196243,5.233635,...,5.318420,5.282375,5.321649,5.207520,2.520922,5.144288,NaN,5.197130,5.333838,5.247336
20170105,5.340569,5.090387,5.120245,4.857066,4.961166,5.217625,5.254137,5.023852,5.066368,5.114312,...,5.266538,5.168544,5.267949,5.053979,2.677950,5.174250,NaN,5.285300,5.387275,5.185889
20170106,5.378719,5.144450,5.252108,5.107856,5.248108,4.743682,5.211414,5.218668,5.077592,5.097054,...,5.393343,5.251686,5.170001,5.073282,3.347747,5.158319,NaN,5.274073,5.342754,5.206626
20170109,5.232279,5.011768,5.125947,5.186148,5.244179,NaN,5.189513,5.264544,5.147670,5.226838,...,5.226348,5.092289,5.268018,5.040950,3.269103,4.935069,NaN,5.279723,5.232351,5.227412
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20210525,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210526,5.290984,4.730743,4.867360,5.193463,5.101544,5.040898,5.053869,5.298752,5.143490,4.821851,...,5.229484,4.909035,5.077453,5.094397,5.046268,4.944638,NaN,4.994130,5.220599,5.289053
20210527,5.198508,4.614271,4.869342,5.241143,5.184105,5.142822,5.057104,5.286123,5.105527,4.934832,...,5.234300,4.908083,5.044460,5.040774,5.215891,5.062917,NaN,4.854092,5.174500,5.208394
20210528,5.149977,4.565239,4.942493,5.252698,5.245510,5.168844,5.063554,5.183503,5.141610,4.367995,...,5.236158,4.585673,5.077039,4.922337,5.131924,4.902078,NaN,4.824750,5.197615,5.207183


In [38]:
tmp.index

Int64Index([20170103, 20170104, 20170105, 20170106, 20170109, 20170110,
            20170111, 20170112, 20170113, 20170116,
            ...
            20210518, 20210519, 20210520, 20210521, 20210524, 20210525,
            20210526, 20210527, 20210528, 20210531],
           dtype='int64', length=1071)

In [39]:
tmp.columns

Index(['600373.SH', '300557.SZ', '000416.SZ', '002156.SZ', '600500.SH',
       '002123.SZ', '000610.SZ', '601699.SH', '603336.SH', '000663.SZ',
       ...
       '000581.SZ', '000606.SZ', '600335.SH', '600010.SH', '300582.SZ',
       '600527.SH', '000699.SZ', '002011.SZ', '600129.SH', '002233.SZ'],
      dtype='object', length=4499)

In [42]:
data = cleaned_data.copy()
data.index

MultiIndex([(600000,  931),
            (600000,  932),
            (600000,  933),
            (600000,  934),
            (600000,  935),
            (600000,  936),
            (600000,  937),
            (600000,  938),
            (600000,  939),
            (600000,  940),
            ...
            (603999, 1451),
            (603999, 1452),
            (603999, 1453),
            (603999, 1454),
            (603999, 1455),
            (603999, 1456),
            (603999, 1457),
            (603999, 1458),
            (603999, 1459),
            (603999, 1500)],
           names=['stock_code', 'time_str'], length=347760)

In [43]:
def cal_Rvar(sample_data): 
    factor = np.sum(sample_data['Return']**2)
    return factor


In [45]:
fac = data.groupby(level=0).apply(cal_Rvar)
fac.index = [str(x)+".SH" for x in fac.index]
fac.name = int(date_csv.split(".")[0])
fac

600000.SH    0.000762
600004.SH    0.000510
600006.SH    0.001735
600007.SH    0.000623
600008.SH    0.001236
               ...   
603993.SH    0.002736
603996.SH    0.000787
603997.SH    0.000519
603998.SH    0.002267
603999.SH    0.001851
Name: 20190226, Length: 1449, dtype: float64

In [34]:
sample_data = cleaned_data.loc[600000]
sample_data.to_csv(r"D:\02PHBS_G2\PHBS_m2\bigDataAnalysis\big-data-hft\project\src\main\python\factorCalFunc\mins_data_1ticker.csv")

In [14]:
process_mr_data(os.path.join(ROOT,"data/20190102.csv"))

feature_name            Amount  Close  Count  High   Low  Open       Qty  \
stock_code time_str                                                        
600000     0931      1463246.0   9.76   72.0  9.78  9.76  9.77  149800.0   
           0932      1199586.0   9.77   77.0  9.79  9.76  9.78  122700.0   
           0933      4607750.0   9.73  236.0  9.78  9.73  9.78  472500.0   
           0934      1260563.0   9.73   80.0  9.76  9.73  9.75  129400.0   
           0935      1112431.0   9.74   76.0  9.76  9.73  9.74  114200.0   
...                        ...    ...    ...   ...   ...   ...       ...   
603999     1456       252573.0   4.75   41.0  4.77  4.75  4.76   53100.0   
           1457            0.0   4.75    0.0  4.75  4.75  4.75       0.0   
           1458            0.0   4.75    0.0  4.75  4.75  4.75       0.0   
           1459            0.0   4.75    0.0  4.75  4.75  4.75       0.0   
           1500       130150.0   4.75   28.0  4.75  4.75  4.75   27400.0   

feature_name           Return  
stock_code time_str            
600000     0931     -0.001024  
           0932     -0.001022  
           0933     -0.005112  
           0934     -0.002051  
           0935      0.000000  
...                       ...  
603999     1456     -0.002101  
           1457      0.000000  
           1458      0.000000  
           1459      0.000000  
           1500      0.000000  

[345120 rows x 8 columns]

In [16]:
raw_data = pd.read_csv(os.path.join(ROOT,"data/20190102.csv"), header=None)
raw_data.columns = ['raw']

In [17]:
raw_data_bond = pd.read_csv(os.path.join(ROOT,"data/20190102.csv"), header=None)
raw_data_bond.columns = ['raw']

In [18]:
tmp1 = raw_data_bond["raw"].str.split("#")
raw_data_bond["stock_code"] = tmp1.str[0]
tmp2 = tmp1.str[1].str.split("$")
raw_data_bond["time_str"] = tmp2.str[0].str[8:]
tmp3 = tmp2.str[1].str.split("\t")
raw_data_bond["feature_name"] = tmp3.str[0]
raw_data_bond["feature_value"] = tmp3.str[1].astype("float32")
raw_data_bond

,raw,stock_code,time_str,feature_name,feature_value
0,010107#201901020925$Amount\t82312.0,010107,0925,Amount,82312.000000
1,010107#201901020925$Close\t102.89,010107,0925,Close,102.889999
2,010107#201901020925$Count\t5.0,010107,0925,Count,5.000000
3,010107#201901020925$High\t102.89,010107,0925,High,102.889999
4,010107#201901020925$Low\t102.89,010107,0925,Low,102.889999
...,...,...,...,...,...
2359800,900957#201901021455$Count\t2.0,900957,1455,Count,2.000000
2359801,900957#201901021455$High\t0.618,900957,1455,High,0.618000
2359802,900957#201901021455$Low\t0.618,900957,1455,Low,0.618000
2359803,900957#201901021455$Open\t0.618,900957,1455,Open,0.618000


In [19]:
raw_data = raw_data[raw_data["raw"].str.startswith("6") ]
raw_data

,raw
148239,600000#201901020925$Amount\t1761966.0
148240,600000#201901020925$Close\t9.74
148241,600000#201901020925$Count\t82.0
148242,600000#201901020925$High\t9.74
148243,600000#201901020925$Low\t9.74
...,...
2341586,603999#201901021500$Count\t28.0
2341587,603999#201901021500$High\t4.75
2341588,603999#201901021500$Low\t4.75
2341589,603999#201901021500$Open\t4.75


In [20]:
tmp1 = raw_data["raw"].str.split("#")
raw_data["stock_code"] = tmp1.str[0]
tmp2 = tmp1.str[1].str.split("$")
raw_data["time_str"] = tmp2.str[0].str[8:]
tmp3 = tmp2.str[1].str.split("\t")
raw_data["feature_name"] = tmp3.str[0]
raw_data["feature_value"] = tmp3.str[1].astype("float32")
raw_data

,raw,stock_code,time_str,feature_name,feature_value
148239,600000#201901020925$Amount\t1761966.0,600000,0925,Amount,1761966.00
148240,600000#201901020925$Close\t9.74,600000,0925,Close,9.74
148241,600000#201901020925$Count\t82.0,600000,0925,Count,82.00
148242,600000#201901020925$High\t9.74,600000,0925,High,9.74
148243,600000#201901020925$Low\t9.74,600000,0925,Low,9.74
...,...,...,...,...,...
2341586,603999#201901021500$Count\t28.0,603999,1500,Count,28.00
2341587,603999#201901021500$High\t4.75,603999,1500,High,4.75
2341588,603999#201901021500$Low\t4.75,603999,1500,Low,4.75
2341589,603999#201901021500$Open\t4.75,603999,1500,Open,4.75


In [21]:
data = raw_data.iloc[:,1:]
data

,stock_code,time_str,feature_name,feature_value
148239,600000,0925,Amount,1761966.00
148240,600000,0925,Close,9.74
148241,600000,0925,Count,82.00
148242,600000,0925,High,9.74
148243,600000,0925,Low,9.74
...,...,...,...,...
2341586,603999,1500,Count,28.00
2341587,603999,1500,High,4.75
2341588,603999,1500,Low,4.75
2341589,603999,1500,Open,4.75


In [22]:
min_data = data.pivot(index=['stock_code','time_str'],columns='feature_name',values='feature_value')

In [23]:
min_data

feature_name            Amount  Close  Count  High   Low  Open       Qty
stock_code time_str                                                     
600000     0925      1761966.0   9.74   82.0  9.74  9.74  9.74  180900.0
           0930      2310484.0   9.76  122.0  9.79  9.75  9.75  236500.0
           0931      1463246.0   9.76   72.0  9.78  9.76  9.77  149800.0
           0932      1199586.0   9.77   77.0  9.79  9.76  9.78  122700.0
           0933      4607750.0   9.73  236.0  9.78  9.73  9.78  472500.0
...                        ...    ...    ...   ...   ...   ...       ...
603999     1453        61880.0   4.76   13.0  4.76  4.76  4.76   13000.0
           1454        63349.0   4.77    8.0  4.77  4.76  4.76   13300.0
           1455       204201.0   4.75   36.0  4.76  4.75  4.76   42900.0
           1456       252573.0   4.75   41.0  4.77  4.75  4.76   53100.0
           1500       130150.0   4.75   28.0  4.75  4.75  4.75   27400.0

[313336 rows x 7 columns]

In [107]:
# min_data['min_return'] = min_data['Close']/min_data['Open'] - 1
min_data

feature_name            Amount  Close  Count  High   Low  Open       Qty
stock_code time_str                                                     
600000     0925      1761966.0   9.74   82.0  9.74  9.74  9.74  180900.0
           0930      2310484.0   9.76  122.0  9.79  9.75  9.75  236500.0
           0931      1463246.0   9.76   72.0  9.78  9.76  9.77  149800.0
           0932      1199586.0   9.77   77.0  9.79  9.76  9.78  122700.0
           0933      4607750.0   9.73  236.0  9.78  9.73  9.78  472500.0
...                        ...    ...    ...   ...   ...   ...       ...
603999     1453        61880.0   4.76   13.0  4.76  4.76  4.76   13000.0
           1454        63349.0   4.77    8.0  4.77  4.76  4.76   13300.0
           1455       204201.0   4.75   36.0  4.76  4.75  4.76   42900.0
           1456       252573.0   4.75   41.0  4.77  4.75  4.76   53100.0
           1500       130150.0   4.75   28.0  4.75  4.75  4.75   27400.0

[313336 rows x 7 columns]

In [24]:
min_data.loc["600647"]

feature_name,Amount,Close,Count,High,Low,Open,Qty
time_str,,,,,,,
0930,186343.0,11.86,28.0,12.09,11.86,11.88,15600.0
0931,17775.0,11.83,9.0,11.88,11.83,11.88,1500.0
0932,1189.0,11.89,1.0,11.89,11.89,11.89,100.0
0933,1183.0,11.83,1.0,11.83,11.83,11.83,100.0
0934,27235.0,11.83,9.0,11.86,11.83,11.86,2300.0
...,...,...,...,...,...,...,...
1452,14244.0,11.88,3.0,11.88,11.82,11.82,1200.0
1453,42804.0,11.89,14.0,11.89,11.89,11.89,3600.0
1455,53505.0,11.89,17.0,11.89,11.89,11.89,4500.0


In [25]:
ref_morning_time = pd.date_range("9:31", "11:30", freq="1min").time
ref_afternoon_time = pd.date_range("13:01", "15:00", freq="1min").time

ref_time_arr = np.concatenate([ref_morning_time,ref_afternoon_time])

ref_time_str_list = [x.strftime("%H%M") for x in ref_time_arr]
ref_code_list = min_data.index.unique(0)

ref_code_time_index =pd.MultiIndex.from_product([ref_code_list,ref_time_str_list],names=['stock_code','time_str'])

In [26]:
ref_code_time_index

MultiIndex([('600000', '0931'),
            ('600000', '0932'),
            ('600000', '0933'),
            ('600000', '0934'),
            ('600000', '0935'),
            ('600000', '0936'),
            ('600000', '0937'),
            ('600000', '0938'),
            ('600000', '0939'),
            ('600000', '0940'),
            ...
            ('603999', '1451'),
            ('603999', '1452'),
            ('603999', '1453'),
            ('603999', '1454'),
            ('603999', '1455'),
            ('603999', '1456'),
            ('603999', '1457'),
            ('603999', '1458'),
            ('603999', '1459'),
            ('603999', '1500')],
           names=['stock_code', 'time_str'], length=345120)

In [27]:
min_data_reindex = min_data.reindex(index=ref_code_time_index)

In [28]:
def my_fillna(df):
    df[['Close']] = df[['Close']].fillna(method='ffill')
    df[['Close','High','Low','Open']] = df[['Close','High','Low','Open']].fillna(method='ffill',axis=1)
    df[['Amount','Count','Qty']] = df[['Amount','Count','Qty']].fillna(0)
    return df

In [29]:
min_data_final = min_data_reindex.groupby("stock_code").apply(my_fillna)
min_data_final

feature_name            Amount  Close  Count  High   Low  Open       Qty
stock_code time_str                                                     
600000     0931      1463246.0   9.76   72.0  9.78  9.76  9.77  149800.0
           0932      1199586.0   9.77   77.0  9.79  9.76  9.78  122700.0
           0933      4607750.0   9.73  236.0  9.78  9.73  9.78  472500.0
           0934      1260563.0   9.73   80.0  9.76  9.73  9.75  129400.0
           0935      1112431.0   9.74   76.0  9.76  9.73  9.74  114200.0
...                        ...    ...    ...   ...   ...   ...       ...
603999     1456       252573.0   4.75   41.0  4.77  4.75  4.76   53100.0
           1457            0.0   4.75    0.0  4.75  4.75  4.75       0.0
           1458            0.0   4.75    0.0  4.75  4.75  4.75       0.0
           1459            0.0   4.75    0.0  4.75  4.75  4.75       0.0
           1500       130150.0   4.75   28.0  4.75  4.75  4.75   27400.0

[345120 rows x 7 columns]

In [32]:
min_data_final.to_csv("data/min_data_cleaned/sample_min_data_20190201.csv")

In [30]:
min_data_final['Return'] = min_data_final['Close']/min_data_final['Open'] - 1
gb_ret = min_data_final.groupby("stock_code")['Return']
feature_df = pd.DataFrame(index=min_data_final.index.unique(0))
min_data_final

feature_name            Amount  Close  Count  High   Low  Open       Qty  \
stock_code time_str                                                        
600000     0931      1463246.0   9.76   72.0  9.78  9.76  9.77  149800.0   
           0932      1199586.0   9.77   77.0  9.79  9.76  9.78  122700.0   
           0933      4607750.0   9.73  236.0  9.78  9.73  9.78  472500.0   
           0934      1260563.0   9.73   80.0  9.76  9.73  9.75  129400.0   
           0935      1112431.0   9.74   76.0  9.76  9.73  9.74  114200.0   
...                        ...    ...    ...   ...   ...   ...       ...   
603999     1456       252573.0   4.75   41.0  4.77  4.75  4.76   53100.0   
           1457            0.0   4.75    0.0  4.75  4.75  4.75       0.0   
           1458            0.0   4.75    0.0  4.75  4.75  4.75       0.0   
           1459            0.0   4.75    0.0  4.75  4.75  4.75       0.0   
           1500       130150.0   4.75   28.0  4.75  4.75  4.75   27400.0   

feature_name           Return  
stock_code time_str            
600000     0931     -0.001024  
           0932     -0.001022  
           0933     -0.005112  
           0934     -0.002051  
           0935      0.000000  
...                       ...  
603999     1456     -0.002101  
           1457      0.000000  
           1458      0.000000  
           1459      0.000000  
           1500      0.000000  

[345120 rows x 8 columns]

In [33]:
data = pd.read_csv("data/min_data_cleaned/sample_min_data_20190201.csv")
data

,stock_code,time_str,Amount,Close,Count,High,Low,Open,Qty,Return
0,600000,931,1463246.0,9.76,72.0,9.78,9.76,9.77,149800.0,-0.001024
1,600000,932,1199586.0,9.77,77.0,9.79,9.76,9.78,122700.0,-0.001022
2,600000,933,4607750.0,9.73,236.0,9.78,9.73,9.78,472500.0,-0.005112
3,600000,934,1260563.0,9.73,80.0,9.76,9.73,9.75,129400.0,-0.002051
4,600000,935,1112431.0,9.74,76.0,9.76,9.73,9.74,114200.0,0.000000
...,...,...,...,...,...,...,...,...,...,...
345115,603999,1456,252573.0,4.75,41.0,4.77,4.75,4.76,53100.0,-0.002101
345116,603999,1457,0.0,4.75,0.0,4.75,4.75,4.75,0.0,0.000000
345117,603999,1458,0.0,4.75,0.0,4.75,4.75,4.75,0.0,0.000000
345118,603999,1459,0.0,4.75,0.0,4.75,4.75,4.75,0.0,0.000000


In [36]:
data[data.stock_code==603999].to_csv("data/min_data_cleaned/min_data_ticker_20190201.csv")

In [39]:
sample_data = pd.read_csv("data/min_data_cleaned/min_data_ticker_20190201.csv",index_col=0)
sample_data

,stock_code,time_str,Amount,Close,Count,High,Low,Open,Qty,Return
344880,603999,931,25480.0,4.90,3.0,4.90,4.90,4.90,5200.0,0.000000
344881,603999,932,30379.0,4.90,6.0,4.90,4.89,4.90,6200.0,0.000000
344882,603999,933,60982.0,4.87,19.0,4.89,4.87,4.89,12500.0,-0.004090
344883,603999,934,205613.6,4.85,30.0,4.87,4.85,4.87,42360.0,-0.004107
344884,603999,935,34395.0,4.84,15.0,4.85,4.84,4.85,7100.0,-0.002062
...,...,...,...,...,...,...,...,...,...,...
345115,603999,1456,252573.0,4.75,41.0,4.77,4.75,4.76,53100.0,-0.002101
345116,603999,1457,0.0,4.75,0.0,4.75,4.75,4.75,0.0,0.000000
345117,603999,1458,0.0,4.75,0.0,4.75,4.75,4.75,0.0,0.000000
345118,603999,1459,0.0,4.75,0.0,4.75,4.75,4.75,0.0,0.000000


In [40]:
def cal_ret_stats(data):
    factor = data["Return"].std()
    return factor
cal_ret_stats(data)

0.0012581498761953598

In [126]:
feature_df['RVar'] = gb_ret.apply(lambda se: se.std()**2)
feature_df['RSkew'] = gb_ret.apply(lambda se: se.skew())
feature_df['RKurtosis'] = gb_ret.apply(lambda se: se.kurtosis())
feature_df

,RVar,RSkew,RKurtosis
stock_code,,,
600000,0.000003,-0.306357,2.297922
600004,0.000002,0.219651,0.564757
600006,0.000004,-0.240814,0.076067
600007,0.000002,0.301550,3.296179
600008,0.000005,-0.122341,0.644067
...,...,...,...
603993,0.000005,-0.372919,0.852593
603996,0.000001,0.085093,1.239972
603997,0.000002,-0.128374,7.294807
